---
title: 9.2 Applications
subject:  Symmetric Matrices
subtitle: 
short_title: 9.2 Applications
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: 
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/08_Ch_9_Symmetric_Matrices/102-Apps.ipynb)

{doc}`Lecture notes <../lecture_notes/Lecture 17 - Introduction to Graph Theory and Consensus Protocols.pdf>`

## Reading

Material related to this page, as well as additional exercises, can be

## Learning Objectives

By the end of this page, you should know:
- TODO

## Introduction to Graph Theory and Consensus

Many applications in engineering involve coordinating groups of agents to behave in a desirable way. Examples abound in a wide range of areas:
* Transportation: air traffic control and intelligent transportation systems
* Military: distributed operative imaging and battle space management
* Robotics: coordination and control among swarms (e.g., on Mars/Moon)
* Sensor networks: adaptive ocean sampling, building sensor networks in green buildings
* General networks: communication, power, and supply chain networks

Many, but not all, of these problems can be posed as consensus problems. In this case, we assume there are $n$ agents, with each agent's state $x_i(t) \in \mathbb{R}$ evolving according to the following differential equation:

\begin{equation}
\dot{x}_i = f_i(x_i) + \sum_{j \in N_i} u_i^j(x_i, x_j), \quad i = 1,\ldots,n \tag{*}
\end{equation}

In equation (*):
* the function $f_i: \mathbb{R} \to \mathbb{R}$ defines the dynamics of agent $i$, i.e., how the internal state of agent $i$ evolves in the absence of other agents.
* the neighbor set $N_i$ of agent $i$ are those agents that are directly connected to agent $i$, as specified by a connection graph $G$ with nodes $i = 1,\ldots,n$ corresponding to agents, and edges defining inter-agent communication.
* the coupling functions $u_i^j: \mathbb{R} \times \mathbb{R} \to \mathbb{R}$ define how the state of agent $j$ influences agent $i$'s dynamics, so that the collective state of the agents $x(t) := (x_1(t),\ldots,x_n(t))$ converges to a desired goal state $x^*$.

In many settings of interest, each agent is a regulator that is trying to regulate or estimate a shared value. This is a common model adopted in distributed optimization and learning applications (e.g., to compute an average of sensor measurements or estimate a parameter). In these settings, it makes sense to set the local dynamics function $f_i(x) = 0$, so that an agent's state is determined solely by interactions with its neighbors. Agent $i$'s dynamics then become

\begin{equation}
\dot{x}_i = \sum_{j \in N_i} u_i^j(x_i, x_j), \quad i = 1,\ldots,n \tag{**}
\end{equation}

A common goal for such systems is for all agents to converge to the same value, i.e., that eventually $x_1 = x_2 = \cdots = x_n = m$, for some value $m$. If this happens, system (**) is said to achieve consensus with consensus value $m$.

Studying the behavior of system (\*\*) will require us to bring together all of the ideas that we've seen over the past few lectures on eigenvalues, dynamical systems, and spectral decomposition of symmetric matrices. We start with a summary of relevant facts from graph theory, which will help us model the flow of information in (\*\*).

## A Primer on Graph Theory

We have used graphs before, but we pause now to describe formal linear algebraic representations thereof.

We begin with the basic definition of a graph, which is defined as a pair $G = (V, E)$ that consists of a set of vertices $V$ and a set of edges $E \subseteq V \times V$. A vertex $v \in V$ is a node in the graph, and an edge $e = (v_i, v_j) \in E$ is an arc connecting node $i$ to node $j$. (Note: sometimes this notation is reversed.)

:::{prf:example} An example graph
:label: graphs-ex1

TODO fix

\begin{tikzpicture}[scale=0.7]
\node[draw, circle] (1) at (0,2) {1};
\node[draw, circle] (2) at (-1,0) {2};
\node[draw, circle] (3) at (0,-2) {3};
\node[draw, circle] (4) at (2,-2) {4};
\node[draw, circle] (5) at (3,0) {5};
\node[draw, circle] (6) at (2,2) {6};

\draw[->] (1) -- (6);
\draw[->] (2) -- (1);
\draw[<->] (2) -- (6);
\draw[->] (3) -- (2);
\draw[->] (3) -- (4);
\draw[->] (4) -- (6);
\draw[->] (5) -- (4);
\draw[->] (5) -- (6);
\draw[->] (6) -- (1);

$V = \{1, 2, 3, 4, 5, 6\}$

$E = \{(1,6), (2,1), (2,3), (2,6), (6,2), (3,4),$ 
    $(3,6), (4,3), (4,5), (5,1), (6,1), (6,2), (6,4)\}$

Path $\pi$ with $V_\pi = \{v_5, v_1, v_2, v_6\}$ and 
$E_\pi = \{(v_5,v_1), (v_1,v_2), (v_2,v_6)\}$ (see defn below)
:::

Some important notation and terminology for graphs include:

* The order of a graph is the number of nodes $|V|$
* Nodes $v_i$ and $v_j$ are adjacent if there exists $e = (v_i, v_j) \in E$ (i.e., if node $i$ is directly connected to node $j$ via an edge $e$)
* An adjacent node $v_j$ for a node $v_i$ is called a neighbor of $v_i$
* The set of all neighbors of $v_i$ is denoted $N_i$
* A graph $G$ is called complete if all nodes are adjacent


We will focus our study on the class of undirected graphs, that is graphs satisfying the property that if $e_{ij} \in E$ then $e_{ji} \in E$. In words, this means that all edges are bidirectional: if $v_j$ is a neighbor of $v_i$, then $v_i$ is a neighbor of $v_j$. Although we won't focus on them, a graph that is not undirected is called directed.

A fundamental question associated with the study of the consensus system (**) is under what properties of interaction among agents can we guarantee that consensus is achieved. Intuitively, if each agent "talks to enough" other agents, we expect consensus to happen. This intuition can be formalized through the notion of graph connectivity.

### Connectedness of Graphs

We start by defining a path in $G$, which is a subgraph $\pi = (V_\pi, E_\pi)$, with with distinct nodes $V_\pi = \{v_1, v_2, ..., v_m\}$ and $E_\pi = \{(v_1,v_2),(v_2,v_3),...,(v_{m-1},v_m)\}$. The length of a path $\pi$ is defined as $|E_\pi| = m-1$. For example, in the graph above, the path $\pi$ defined by $V_\pi = \{v_5, v_1, v_6, v_2\}$ and $E_\pi = \{(v_5,v_1),(v_1,v_6),(v_6,v_2)\}$ is the path that goes from node 5 to node 1 to node 6 to node 2. It has length 3, the number of edges traversed. An undirected graph $G$ is called connected if there exists a path $\pi$ between any two distinct nodes of $G$.

:::{prf:example} Connected and disconnected graphs
:label: graphs-ex2

The following (undirected graphs) are connected:

\begin{tikzpicture}[scale=0.5]
\begin{scope}
    \node[draw, circle] (1) at (0,0) {};
    \node[draw, circle] (2) at (-1,1) {};
    \node[draw, circle] (3) at (1,1) {};
    \node[draw, circle] (4) at (-1,-1) {};
    \node[draw, circle] (5) at (1,-1) {};
    \draw (1) -- (2) -- (3) -- (1) -- (4) -- (5) -- (1);
\end{scope}

\begin{scope}[xshift=5cm]
    \node[draw, circle] (1) at (0,2) {};
    \node[draw, circle] (2) at (-1,1) {};
    \node[draw, circle] (3) at (1,1) {};
    \node[draw, circle] (4) at (0,0) {};
    \node[draw, circle] (5) at (-1,-1) {};
    \node[draw, circle] (6) at (1,-1) {};
    \draw (1) -- (2) -- (4) -- (3) -- (1);
    \draw (4) -- (5);
    \draw (4) -- (6);
\end{scope}
\end{tikzpicture}

This graph, composed of two disconnected subgraphs, is not:

\begin{tikzpicture}[scale=0.5]
\node[draw, circle] (1) at (0,0) {};
\node[draw, circle] (2) at (1,1) {};
\node[draw, circle] (3) at (1,-1) {};
\node[draw, circle] (4) at (3,0) {};
\draw (1) -- (2) -- (3) -- (1);
\end{tikzpicture}
:::

## Matrices Associated with a Graph

In our study of network flow problems, we encountered the incidence matrix associated with a graph. However, for our consensus problems, we can associate another matrix with a graph $G$, and while the incidence matrix is convenient for encoding flow conservation, we will see that the Graph Laplacian Matrix is more natural for defining consensus algorithms. We begin by defining the adjacency matrix $A \in \mathbb{R}^{n \times n}$ of graph $G$ of order $n$ by

$$
a_{ij} = \begin{cases}
1 & \text{if } (v_i, v_j) \in E, \quad i,j=1,\ldots,n \\
0 & \text{otherwise}
\end{cases}
$$

The degree matrix $\Delta$ of a graph $G$ is a diagonal $n \times n$ matrix with diagonal elements $\Delta_{ii}$ specified by the number of edges leaving node $i$, also called the out-degree of $v_i$, which we'll denote by out$(v_i)$:

$$
\Delta_{ii} = \text{out}(v_i), \quad i=1,\ldots,n
$$

The Laplacian matrix $L$ of a graph is defined as $L = \Delta - A$. An important property of the Laplacian is that its rows all sum to zero, and that if a graph is undirected, then its adjacency matrix and its Laplacian are both symmetric.

:::{prf:example} Adjacency and Laplacian matrices
:label: graphs-ex3

The adjacency matrix and Laplacian for the directed graph above are:

$$
A = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 \\
1 & 1 & 0 & 1 & 0 & 0
\end{bmatrix}
\quad
L = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 \\
0 & 2 & 0 & -1 & 0 & -1 \\
0 & 0 & 2 & -1 & 0 & -1 \\
0 & 0 & -1 & 2 & -1 & 0 \\
-1 & 0 & 0 & 0 & 1 & 0 \\
-1 & -1 & 0 & -1 & 0 & 3
\end{bmatrix}
$$

:::

## Consensus Protocols 

Consider a collection of $N$ agents that communicate along a set of undirected links described by a graph. Each agent has state $x_i(t) \in \mathbb{R}$, with initial value $x_i(0)$, and together, they wish to determine the average of the initial states $\text{avg}(x(0)) = \frac{1}{N} \sum x_i(0)$.

The agents implement the following consensus protocol:

$$
\dot{x}_i = \sum_{j \in N_i} (x_j - x_i) = -|N_i| (x_i - \text{avg}(x_{N_i})),
$$

where $\text{avg}(x_{N_i}) = \frac{1}{|N_i|} \sum_{j \in N_i} x_j$ is the average of the states of the neighbors of agent $i$. This is equivalent to the first-order homogeneous linear ordinary differential equation:

$$
\dot{x} = -Lx. \quad (\text{AVG})
$$

Based on our previous analysis of such systems, we know that the solution to (AVG) is given by

$$
x(t) = c_1 e^{\lambda_1 t} v_1 + \cdots + c_n e^{\lambda_n t} v_n, \quad x(0) = \sum v_i. \quad (\text{SOL})
$$

where $(\lambda_i, v_i)$, $i=1,\ldots,n$, are the eigenvalue/eigenvector pairs of the negative graph Laplacian $-L$. Thus, the behavior of the consensus system (AVG) is determined by the spectrum of $L$. We will spend the rest of today's lecture on understanding the following theorem:

:::{prf:example} Consensus theorem
:label: consensus-thm

If the graph $G$ defining the consensus system (AVG) is connected, then the state of the agents converges to $x^* = \text{avg}(x(0))$ exponentially quickly.

:::

This result is extremely intuitive! It says that so long as the information at one node can eventually reach every other node in the graph, then we can achieve consensus via the protocol (AVG). Let's try to understand why. As in last class we order the eigenvalues of $-L$ in decreasing order: $\lambda_1 \geq \lambda_2 \geq \cdots \geq \lambda_n$.

Our first observation is that $\lambda_1 = 0$, $v_1 = \frac{1}{\sqrt{N}} \mathbf{1}$ is an eigenvalue/eigenvector pair for $-L$. This follows from the fact that each row of $L$ sums to 0, and so:

$$
-L \mathbf{1} = \mathbf{0} = 0 \mathbf{1}.
$$

A fact that we'll show is true later is that the eigenvalues of $L$ are all nonnegative, and thus we know that $\lambda_1 \leq 0$ for the eigenvalues of $-L$. As such, we know that $\lambda_1 = 0$ is the largest eigenvalue of $-L$. Leave we label them $\lambda_1 = 0$, $v_1 = \frac{1}{\sqrt{N}} \mathbf{1}$.

Next, we recall that for an undirected graph, the Laplacian $L$ is symmetric, and hence is diagonalized by an orthonormal eigenbasis $-L = Q \Lambda Q^T$, where $Q = [v_1 \cdots v_n]$ is an orthogonal matrix composed of orthonormal eigenvectors of $L$, and $\Lambda = \text{diag}(\lambda_1, \ldots, \lambda_n)$. Although we do not know $v_2, \ldots, v_n$, we know that $v_1 = \frac{1}{\sqrt{N}} \mathbf{1}$.

We can therefore rewrite (SOL) as:

$$
\begin{aligned}
x(t) &= c_1 e^{0t} \frac{1}{\sqrt{N}} \mathbf{1} + c_2 e^{\lambda_2 t} v_2 + \cdots + c_n e^{\lambda_n t} v_n \\
&= c_1 \frac{1}{\sqrt{N}} \mathbf{1} + c_2 e^{\lambda_2 t} v_2 + \cdots + c_n e^{\lambda_n t} v_n \quad (*)
\end{aligned}
$$

where now we can compute $c$ by solving $x(0) = Qc \Rightarrow c = Q^T x(0)$, as $Q$ is an orthogonal matrix.

Let's focus on computing $c_1$:

$$
c_1 = v_1^T x(0) = \frac{1}{\sqrt{N}} \mathbf{1}^T x(0) = \frac{1}{\sqrt{N}} \sum_{i=1}^N x_i(0).
$$

Plugging this back into (*), we get:

$$
\begin{aligned}
x(t) &= \frac{1}{N} \sum_{i=1}^N x_i(0) \cdot \mathbf{1} + c_2 e^{\lambda_2 t} v_2 + \cdots + c_n e^{\lambda_n t} v_n \\
&= \text{avg}(x(0)) \mathbf{1} + c_2 e^{\lambda_2 t} v_2 + \cdots + c_n e^{\lambda_n t} v_n \quad (**).
\end{aligned}
$$

This is very exciting! We have shown that the solution $x(t)$ to (AVG) is composed of a sum of the final consensus state $x^* = \text{avg}(x(0)) \mathbf{1}$ and exponential functions $c_i e^{\lambda_i t} v_i$, $i=2,\ldots,n$, existing in the subspace $v_1^\perp$ orthogonal to the consensus direction $\frac{1}{\sqrt{N}} \mathbf{1}$. Thus, if we can show that $\lambda_2, \ldots, \lambda_n < 0$, we will have established our result.

To establish this result, we start by stating a widely used theorem for bounding localizing eigenvalues.

:::{prf:example} Gershgorin's disk theorem
:label: gershgorin-thm

Let $A \in \mathbb{R}^{n \times n}$ and define the radius
$$
r_i = \sum_{j=1, j \neq i}^n |a_{ij}|
$$
as the absolute row sum with entry $a_{ii}$ deleted. Then all eigenvalues of $A$ are located in the union of $n$ disks:
$$
\sigma(A) = \bigcup_{i=1}^n G_i(A), \quad G_i(A) = \{z \in \mathbb{C} \mid |z - a_{ii}| \leq r_i\}
$$

In the case of symmetric matrices, we can restrict the $G_i(A)$ to the real line:
$$
G_i(A) = \{\lambda \in \mathbb{R} \mid |\lambda - a_{ii}| \leq r_i\}
$$
:::

:::{prf:example} Gershgorin's disk theorem
:label: graphs-ex4

Consider $A = \begin{bmatrix} 3 & 1 \\ 1 & 3 \end{bmatrix}$. Gershgorin's disk theorem tells us that the eigenvalues $\lambda_1$ and $\lambda_2$ are contained within the set
$$
G(A) = \{\lambda \in \mathbb{R} \mid |\lambda - 3| \leq 1\},
$$
or equivalently, that $2 \leq \lambda_2 \leq \lambda_1 \leq 4$. As we've computed in previous examples, $\lambda_1 = 4$ and $\lambda_2 = 2$, which indeed do lie within $G(A)$.

:::

Let's apply this theorem to a graph Laplacian $L$. The diagonal elements of $L$ are $\Delta = A_{ii} = \text{out}(v_i)$, the out-degree of node $i$. Further, the row $r_i = \text{out}(v_i)$ as well, as $a_{ij} = -1$ if node $i$ is connected to node $j$, and 0 otherwise. Therefore, for row $i$, we have the following Gershgorin intervals:
$$
G_i(L) = \{\lambda \in \mathbb{R} \mid |\lambda - \text{out}(v_i)| \leq \text{out}(v_i)\}.
$$

These are intervals of the form $[0, 2\text{out}(v_i)]$, and therefore the union $G(L) = \bigcup_{i=1}^n G_i(L) = [0, 2d_{\text{max}}]$, where $d_{\text{max}} = \max_i \text{out}(v_i)$ is the maximum out degree of a node in the graph. Taking the negative of everything, we conclude that $G(-L) = [-2d_{\text{max}}, 0]$.

This tells us that $\lambda_i \leq 0$ for $i=1,2,\ldots,n$ for the eigenvalues of $-L$. This is almost what we wanted. We still need to show that only $\lambda_1 = 0$ and that $\lambda_n \leq \cdots \leq \lambda_2 < 0$. To answer this question, we rely on the following proposition:

:::{prf:proposition} Connected components and zero eigenvalues
:label: connected-components-prop

The algebraic multiplicity of the 0 eigenvalue of a graph Laplacian $L$ is equal to the number of connected components in the graph. In particular, if the graph $G$ is connected, then only $\lambda_1 = 0$, and $\lambda_n \leq \lambda_2 < 0$.

:::

Unfortunately proving this result would take us too far astray. Instead, we highlight the intuitive nature of the result in terms of the consensus system (AVG). This proposition tells us that if the communication graph $G$ is strongly connected, i.e., if it requires information to eventually reach every node, then $x(t) \to \text{avg}(x(0))\mathbf{1}$ is guaranteed by the slowest decaying mode $e^{\lambda_2 t}$.

In contrast, suppose the graph $G$ is disconnected, and consists of the disjoint union of two connected graphs $G_1 = (V_1, E_1)$ and $G_2 = (V_2, E_2)$, i.e., $G = G_1 \cup G_2$ and $V_1 \cap V_2 = \emptyset$ and $E_1 \cap E_2 = \emptyset$. Then if we run the consensus protocol (AVG) on $G$, the states of agents in $G_1$ and $G_2$ will evolve to two parallel states, each acting on their own graph completely unaware of the other:

\begin{equation}
\dot{x}_1 = -L_1 x_1 \quad \text{and} \quad \dot{x}_2 = -L_2 x_2, \tag{*}
\end{equation}

Here we use $x_i$ to denote the state of agents in $G_i$, with Laplacian $L_i$, and similarly for $x_2$. By the above discussion, if $L_1$ and $L_2$ are both strongly connected, then $x_1(t) \to \text{avg}(x_1(0)) \mathbf{1}$, and $\lambda = 0, v = \mathbf{1}$ is an eigenvalue/vector pair for each graph.

If we now consider the joint graph $G$ composed of the two disjoint graphs $G_1$ and $G_2$, we can immediately see how to change our consensus protocol:
$\dot{x} = -Lx$ will evolve as it did before.

To see how this manifests in the algebraic multiplicity of the 0 eigenvalue of $L = \begin{bmatrix} L_1 & \\ & L_2 \end{bmatrix}$, note that for the composite system with state $x = \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$, we have the consensus dynamics:

$$
\begin{bmatrix} \dot{x}_1 \\ \dot{x}_2 \end{bmatrix} = \begin{bmatrix} -L_1 & \\ & -L_2 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix},
$$

which has $\lambda_1 = 0$ with $v_1 = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$ and $\lambda_2 = 0$ with $v_2 = \begin{bmatrix} 0 \\ 1 \end{bmatrix}$ so that:

$$
\begin{bmatrix} x_1^* \\ x_2^* \end{bmatrix} = \begin{bmatrix} 1 \\ 0 \end{bmatrix} \text{avg}(x_1(0)) + \begin{bmatrix} 0 \\ 1 \end{bmatrix} \text{avg}(x_2(0)).
$$

This is of course expected, as all we have done is rewrite (*) using block vectors and matrices—we have not changed anything about the consensus protocol.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/08_Ch_9_Symmetric_Matrices/102-Apps.ipynb)